# SWE bench Solver Agentic Workflow

## Requirements

In [23]:
!pip install -q astropy gitpython langchain langchain_openai langchain_community \
    hypothesis langchain_groq langgraph pytest typing_extensions langchain-experimental datasets \
    Pygments black unidiff patch ray filelock transformers "vLLM>=0.3.0" "accelerate>=0.26.0"
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Sat Jan 25 15:27:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:00:07.0 Off |                    0 |
| N/A   47C    P0             73W /  300W |   80777MiB /  81920MiB |      0%      Default |
|                

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Imports

In [2]:
from langgraph.graph import StateGraph, END, MessagesState, add_messages
from typing import TypedDict, Annotated, Sequence
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, FunctionMessage, AnyMessage, ToolMessage
from langgraph.prebuilt import ToolExecutor, ToolInvocation, ToolNode
from langchain_core.runnables import RunnableConfig
import inspect, os
from typing import Any, Dict, Callable, List,Tuple
import git
from langchain_community.tools import ShellTool, tool
from pydantic import BaseModel

from time import time
import re, operator, json, git
from datasets import load_dataset
from pprint import pprint
import sys
import subprocess
import tempfile
import yaml
import warnings
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import shutil


from pydantic import BaseModel
OPENAI_API_KEY = ""
GROQ_API_KEY = ''
# Suppress all warnings
warnings.filterwarnings("ignore")

## VLLM

In [25]:
# from transformers import AutoTokenizer
# from vllm import LLM, SamplingParams
# from huggingface_hub import hf_hub_url, snapshot_download,login

# # Clear cache manually
# !rm -rf ~/.cache/huggingface/hub
# login(token="")


# # Initialize the tokenizer for Llama 2
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# # Define sampling parameters
# sampling_params = SamplingParams(
#     temperature=0.7,
#     top_p=0.8,
#     repetition_penalty=1.05,
#     max_tokens=512
# )

# # Initialize the LLM with the Llama 2 model
# llm = LLM(model="meta-llama/Llama-2-7b-chat-hf")


# Quen Model
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/QwQ-32B-Preview")

sampling_params = SamplingParams(
    temperature=0.6,
    top_p=1.0,
    repetition_penalty=1.0,
    max_tokens=4000
)

llm = LLM(
    model="Qwen/QwQ-32B-Preview",
    max_model_len=4000
)



INFO 01-25 15:30:11 config.py:510] This model supports multiple tasks: {'reward', 'embed', 'score', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 01-25 15:30:11 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='Qwen/QwQ-32B-Preview', speculative_config=None, tokenizer='Qwen/QwQ-32B-Preview', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/QwQ-32B-Preview, 

OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 79.25 GiB of which 124.75 MiB is free. Process 4105875 has 79.12 GiB memory in use. Of the allocated memory 78.21 GiB is allocated by PyTorch, with 68.00 MiB allocated in private pools (e.g., CUDA Graphs), and 130.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## LLM Tool Nodes

In [33]:
shell = ShellTool()
abs_project_dir = os.getcwd()


# -------------------------------------------------------------------------------------------------------------------------------
class CurrentDirectory:
    cwd: str = '/'

directory = CurrentDirectory()
# -------------------------------------------------------------------------------------------------------------------------------
# tools Binder
def getSearchTools():
    return [ls, goto_directory, goto_previous_dir, get_current_dir, number_of_lines, open_file, find_files, search_file, search_dir]
def getContentViewingTools():
    return [get_files_content]
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def ls() -> str:
    """
    This function lists the files in the current directory.
    :function: ls
    :return: ->  a string containing the path of current directory and -> the files present in the current directory
    """
    print(f"Listing files in current directory: {directory.cwd}")
    ls_out = shell.run({"commands": [f"cd test-repos{directory.cwd}", 'ls']})
    return f"""
Current Directory: {directory.cwd}
Files:
{ls_out}
"""
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def goto_directory(path: str) -> str:
    """
    This function changes the current directory to the specified directory. This function must be used to goto a directory not to open a file.
    :function: goto_dir
    :param str path: path of the new directory you want to change relative to the current directory e.g 'matplotlib/doc'
    :return: output of command 'fail' or 'success'
    """
    print(f"Attempting to change directory to: {path}")
    if '..' in path:
        return 'use goto_previous_dir tool instead to go to previous directory'
    out = shell.run({"commands": [f"cd test-repos{directory.cwd}", f'cd {path}']})
    if out == "":
        directory.cwd = f"{directory.cwd}{path}/"
        print(f"Successfully entered {directory.cwd}")
        return 'successfully entered ' + directory.cwd
    else:
        return out
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def goto_previous_dir() -> str:
    """
    This function takes the user to the previous directory.
    :function: goto_previous_dir
    :return: output
    """
    print("Attempting to go to previous directory")
    if directory.cwd == '/':
        return "Already in top most directory. Can't go back anymore"
    else:
        paths = directory.cwd.split('/')[1:-1]
        paths.pop()
        directory.cwd = '/'
        for dir in paths:
            directory.cwd += dir + '/'
        return f"Current Directory: {directory.cwd}"
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def get_current_dir() -> str:
    """
    This function returns the path of the currently opened directory.
    :function: get_current_dir
    :return: the current directory
    """
    return directory.cwd
# -------------------------------------------------------------------------------------------------------------------------------
def get_abs_current_dir() -> str:
    return os.path.join(abs_project_dir, 'test-repos', directory.cwd[1:-1])
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def number_of_lines(path: str) -> str:
    """
    This function takes a file path as input and returns the number of lines in the file.
    :function: number_of_lines
    :param path: The relative path to the file (e.g., 'lib/matplotlib/axis.py').

    :return: The number of lines in the file.
    """

    abs_file_path = os.path.join(get_abs_current_dir(), path)
    if os.path.exists(abs_file_path):
        with open(abs_file_path, 'r') as file:
            return f"Number of lines in {directory.cwd+path}: {sum(1 for line in file)}"
    else:
        return f"File {directory.cwd+path} not found"
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def open_file(path: str, line_number: int = 1, max_lines: int = 100) -> str:
    """
    This function takes a file path, a line number, and a maximum number of lines as
    input and returns the contents of the file starting from the specified line number, limited to the maximum number of lines.
    :function: open_file

    :param path: The relative path to the file (e.g., 'lib/matplotlib/axis.py').
    :param line_number: The line number from which to start reading the file. Defaults to 1.
    :param max_lines: The maximum number of lines to return from the starting line. Defaults to 100.
    :return: A string containing the file contents from the specified starting line, limited to max_lines.

    """

    if line_number < 1:
        return "Error: line number cannot be zero or negative"
    if max_lines < 1:
        return "Error: max_lines cannot be zero or negative"

    print(f"Opening file: {path} from line {line_number} with max {max_lines} lines")
    abs_file_path = os.path.join(get_abs_current_dir(), path)
    if os.path.exists(abs_file_path):
        with open(abs_file_path, 'r') as file:
            with open(abs_file_path, 'r') as temp_file:
                num_lines = sum(1 for line in temp_file)
            if line_number > num_lines:
                print(f"Line {line_number} exceeds file length {num_lines}")
                return f"Can't access {line_number} line. This file only contains {num_lines} lines"

            out = f"Showing contents of File: {directory.cwd+path} starting from {line_number}\n\n"
            for n, line in enumerate(file, 1):
                if n >= line_number:
                    out += f"{n}: {line}\n"
                    if n == line_number + max_lines - 1:
                        break
            return out
    else:
        return path + " doesn't exist"
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def find_files(file_name: str) -> str:
    """
    Searches the current directory and its subdirectories for the files that have name containing the specified file_name.
    :function: find_files
    :param file_name: The file_name to search for in file names.
    :return: paths of the files that contain the specified file_name in their names.
    """
    matched_files = []

    # Walk through the current directory and all subdirectories
    for root, dirs, files in os.walk("test-repos" + directory.cwd):
        for file in files:
            if file_name in file:
                matched_files.append(os.path.join(directory.cwd, file))

    return "Files found:\n" + "\n".join(matched_files)
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def search_file(path: str, search_term: str) -> str:
    """
    This function takes a file path and a search term as input and returns the lines in the file that contain the search term.
    :param str path: The relative path to the file (e.g., 'lib/matplotlib/axis.py').
    :param str search_term: The term to search for in the file.
    :return: A string containing the lines in the file that contain the search term.
    """

    abs_file_path = os.path.join(get_abs_current_dir(), path)
    if os.path.exists(abs_file_path):
        with open(abs_file_path, 'r') as file:
            out = f"Searching for '{search_term}' in {directory.cwd+path}\n\n"
            for n, line in enumerate(file, 1):
                if search_term in line:
                    out += f"{n}: {line}\n"
            return out
    else:
        return f"File {directory.cwd+path} not found"

# -------------------------------------------------------------------------------------------------------------------------------
@tool
def search_dir(path: str, search_term: str) -> str:
    """
    Searches for files in the specified directory that contain the search term. It returns the file names and line numbers where the search term is found.

    :param path: The relative path to the directory (e.g., 'lib/matplotlib').
    :param search_term: The term to search for in the files in the directory.
    :return: A string containing the files in the directory that contain the search term.
    """
    abs_dir_path = os.path.join(get_abs_current_dir(), path)

    if os.path.exists(abs_dir_path):
        matched = ""
        for root, dirs, files in os.walk(abs_dir_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file_a:
                        for n, line in enumerate(file_a, 1):
                            if search_term in line:
                                matched += f"File: {file_path}, Line: {n}\n"
                                break
                except (UnicodeDecodeError, IOError):  # Handle decoding errors and file I/O errors
                    continue

        if matched:
            return "Files found:\n" + matched
        else:
            return "No files containing the search term were found."
    else:
        return f"Directory {abs_dir_path} not found."
# -------------------------------------------------------------------------------------------------------------------------------
@tool
def get_files_content(paths: List[str], line_numbers: List[int]) -> str:
    """
    This function takes a list of file paths and a list of line numbers as input and returns the contents of the files starting from the specified line numbers.
    :param paths: A list of relative paths to the files (e.g., ['lib/matplotlib/axis.py', 'lib/matplotlib/figure.py']).
    :param line_numbers: A list of line numbers from which to start reading the files.
    :return: A string containing the contents of the files from the specified starting line numbers.
    """
    out = ""
    for path, line_number in zip(paths, line_numbers):
        out += open_file.invoke({'path': path, "directory.cwd": directory.cwd, 'line_number': line_number, 'max_lines': 50}) + '\n\n'
    return out

# -------------------------------------------------------------------------------------------------------------------------------



## Repo Cloner

In [34]:

# def setup_repo(data: dict):
#     repo_url = f"https://github.com/{data['repo']}.git"
#     repo_name = data['repo'].split('/')[-1]
#     test_repos_dir = "test-repos"
#     repo_dir = os.path.join(test_repos_dir, repo_name)

#     if not os.path.exists(test_repos_dir):
#         os.makedirs(test_repos_dir)
#         print(f"Created directory {test_repos_dir}")

#     os.chdir(test_repos_dir)
    
#     if not os.path.exists(repo_name):
#         print(subprocess.run(["git", "clone", repo_url], capture_output=True, text=True).stdout)
#     else:
#         os.chdir(repo_name)
#         # Stash any local changes, pull the latest updates, and reset the branch
#         print(subprocess.run(["git", "stash"], capture_output=True, text=True).stdout)
#         print(subprocess.run(["git", "pull"], capture_output=True, text=True).stdout)
#         print(subprocess.run(["git", "reset", "--hard", "origin/" + data['base_branch']], capture_output=True, text=True).stdout)

#     # Checkout to a new branch based on the specified commit
#     result = subprocess.run(["git", "checkout", "-b", data['instance_id'], data['base_commit']], capture_output=True, text=True)
#     print(result.stdout if result.stdout else result.stderr)
#     print('Repo setup complete.')



# def setup_repo(data: dict):
#     repo_name = data['repo'].split('/')[-1]
#     directory.cwd = f"/{repo_name}/" 
#     if not os.path.exists("test-repos"):
#         print(shell.run({"commands": [
#             "mkdir test-repos",
#         ]}))
#     if not os.path.exists(f"test-repos/{repo_name}"):
#         # git.Git("test-repos").clone(f"https://github.com/{data['repo']}.git")
#         print(shell.run({"commands": [
#             f"cd test-repos",
#             f"git config --global http.postBuffer 524288000",
#             f"git clone https://github.com/{data['repo']}.git"
#         ]}))
#     print(shell.run({"commands": [
#         f"cd test-repos/{repo_name}",
#         # f"pip install -e .",
#         f"git checkout -b {data['instance_id']} {data['base_commit']}",
#     ]}))
#     print('Repo setup complete')



def setup_repo(data: dict):
   repo_name = data['repo'].split('/')[-1]
   directory.cwd = f"/{repo_name}/"
   
   if not os.path.exists("test-repos"):
       print(shell.run({"commands": ["mkdir test-repos"]}))
       
   if os.path.exists(f"test-repos/{repo_name}"):
       print(shell.run({"commands": [
           f"cd test-repos/{repo_name}",
           "git fetch origin",
           "git reset --hard origin/main",
           "git clean -fd"
       ]}))
   else:
       print(shell.run({"commands": [
           "cd test-repos",
           "git config --global http.postBuffer 524288000",
           f"git clone https://github.com/{data['repo']}.git"
       ]}))
       
   print(shell.run({"commands": [
       f"cd test-repos/{repo_name}",
       f"git checkout -b {data['instance_id']} {data['base_commit']}"
   ]}))
   
   print('Repo setup complete')

    

## Graph State & llms

In [35]:

class Diagnosis(TypedDict):
    file: str
    affected_functions: List[Dict[str, Any]] 
    recommended_fix_strategy: List[str]  


class PatchState(TypedDict):
    patch: str
    code: str
    example_patch: str
    data: dict
    repo: str
    patch_application_failures: int
    messages: Annotated[list[AnyMessage], add_messages]
    diagnosis: Diagnosis  
    failed_tests: List[str]



def display_patch_state(state: PatchState) -> None:
    print("\n--- Patch State Details ---")
    for key, value in state.items():
        print(f"{key.capitalize()}:")
        pprint(value, width=80)  
        print()  



# model = "gpt-4o-mini"
model = "gpt-4o"
diagnoser_llm = ChatOpenAI(
    model=model,
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY, 
)
extractor_llm = ChatOpenAI(
    model=model,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY, 
)
search_llm = ChatOpenAI(
    model=model,
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY,
)
editor_llm = ChatOpenAI(
    model=model,
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY,
)

test_runner_llm = ChatOpenAI(
    model=model,
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_API_KEY, 
) 

# Bind tools for file viewing
# editor_llm = editor_llm.bind_tools([open_file, get_current_dir, number_of_lines])
diagnoser_llm = diagnoser_llm.bind_tools(getSearchTools())



search_tool_node = ToolNode(tools=getSearchTools())
extract_code_tool_node = ToolNode(tools=getContentViewingTools())

## Prompt Templates

In [36]:

# search_issue_code_prompt = PromptTemplate.from_template("""
# You are tasked with diagnosing an issue in the {repo} repository. Currently, you are in the {cwd} directory. Below is the problem statement describing the issue:
# Problem Statement: {issue}
# Your objective is to identify the specific files and line numbers in the repository that are likely causing this issue.
# pelase make sure code issue dont overlap and are precise and short non repeating
# Guidelines:
# 1. Output your findings in one of the following in json format:
#    - File: <file_path> Function: <function_name> Line Range: <start_line> to <end_line> - <description>
#    - File: <file_path> Line Range: <start_line> to <end_line> - <description>
#    - File: <file_path> Line Range: <start_line> to <end_line> - <description> (chunk)
   
# 2. Do not attempt to resolve the issue; only identify the potential causes.

# 3. the Line range must be accurate to sections please start from define maximum 1 fucntion in one finding 
# Output the most relevant results for the problem in the format above. Allow the next agent to continue the analysis as needed.
# """)


search_issue_code_prompt = PromptTemplate.from_template("""
You are tasked with identifying problematic functions in a Python codebase. Below is the description of the issue:

---
{issue}
---

Identify the specific **functions** causing the issue. Provide the following details for each problematic function:
1. File: <file_path> (relative file path like "astropy/io/ascii/qdp.py") 
2. Function: <function_name>
3. Line Range: <start_line> to <end_line> (Include the entire function definition, including decorators and docstrings.)
4. Reasoning: Why this function is likely related to the issue.

Ensure the **line range includes the entire function definition** and does not overlap with other functions.
If multiple functions are likely problematic, list each separately.


No description or explaination required please return in a json single code block only

""")


extract_content_from_files_prompt = PromptTemplate.from_template("""
These are the files with lines of code. 
{files}
                                                                 
Use the tools provided to you to provide the contents of these files strating from the line numbers. Remember currently you are in {current_dir} directory.                                                              
""")



editor_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a code editor tasked with analyzing issues and suggesting code fixes."),
    ("human", """
File Path: {file_path}
Current Directory: {current_dir}
Line Range: {line_range}
Issue Description: {issue_description}

The file content is:
{original_code}


Required Response Format:
```python
20: def example():  # Must keep original line numbers
21:     # Must preserve exact indentation
```

Key Guidelines:
1. Code block must match exact line numbers
2. Preserve all whitespace/indentation
3. Keep code block clean - no comments or explanations inside

""")
])


# Define the LLM prompt template for generating standalone code
test_prompt_template = ChatPromptTemplate.from_template("""
Given the following test content, resolved file content, and the failed function, create a standalone Python script:
Test Path: {test_path}
Failed Function: {failed_function}
Test Content:
{test_content}

Fixed File Content:
{fixed_content}

The standalone script should include:
1. The reimplemented function ({failed_function}) that addresses the issues in the test.
2. A rewritten test function for the reimplemented function.
3. A main function to execute the test function.
Ensure no dependencies or bindings to the original repository.

Output format:
{{
    "standalone_script": "The complete standalone script"
}}
""")

from typing import Dict, Any, List
from dataclasses import dataclass
from langchain.schema import HumanMessage

@dataclass
class DiagnosisMessages:
    FORMAT_REMINDER: str = """Please provide the diagnosis in the following format:
1. **File**: `<file_path>`
   **Function**: `<function_name>`
   **Line Range**: <start> to <end>
   **Reasoning**: <clear explanation>
   
Please ensure all fields are properly formatted and complete."""

    ANALYSIS_REQUEST: str = """Analyze the code and identify specific problematic functions.
For each function include:
- File location
- Function name
- Line range
- Clear reasoning for why it's problematic"""

def create_retry_message() -> HumanMessage:
    return HumanMessage(content=DiagnosisMessages.FORMAT_REMINDER)



def prepare_llm_message(file_path: str, current_dir: str, line_range: str, issue_description: str, original_code: str) -> list:
    """Prepares message content for LLM input."""
    return [
        {"role": "system", "content": """You are a code editor . Your task is to suggest precise, production-ready code fixes.
Key requirements:
Read from hte issue description and then see the original code and suggest me a code block, no explaiantions needed

- Preserve exact line numbers and indentation
- Only output executable code within code blocks
- Focus on practical, maintainable solutions"""},
        {"role": "user", "content": f"""Review and fix the following code issue:

File: {file_path}
Directory: {current_dir}
Line Range: {line_range}
Issue Description: {issue_description}

Original Code:
{original_code}

Required Response Format:
```python
20: def example():  # Must keep original line numbers
21:     # Must preserve exact indentation
```

Key Guidelines:
1. Code block must match exact line numbers
2. Preserve all whitespace/indentation
3. Keep code block clean - no comments or explanations inside
"""}
    ]
    

## Helpers

In [37]:
def parse_response_to_diagnosis(response: str) -> Dict[str, Any]:
    diagnosis = {
        "file_groups": {}  # Group files by file name with their respective line ranges
    }

    # Remove code block markers (```json``` and closing ```) if present
    response = response.strip()

    # Remove the ```json``` and ``` markers
    if response.startswith('```json'):
        response = response[len('```json'):].strip()
    if response.endswith('```'):
        response = response[:-3].strip()

    # Check if the response is empty after stripping
    if not response:
        print("Warning: The response is empty or only contains whitespace.")
        return diagnosis

    # Parse the response as JSON
    try:
        response_json = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print("Response content:", response)
        return diagnosis

    # Grouping by file
    for entry in response_json:
        file = entry.get("File")  # Using the correct key "File"
        line_range = entry.get("Line Range")  # Using the correct key "Line Range"
        description = entry.get("description")  # Using the correct key "description"

        if file:
            if file not in diagnosis["file_groups"]:
                diagnosis["file_groups"][file] = []
            diagnosis["file_groups"][file].append({
                "lines": line_range,
                "issue": description
            })

    return diagnosis




def run_command(command: str, code: str, options: list = None):
    try:
        print(f"Running command: {command} with options: {options}")
        print(f"Code to be executed:\n{code}")
        print(f"~---------------------------------------------~")

        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(code.encode())
            temp_file_path = temp_file.name

        cmd = [command] + (options or []) + [temp_file_path]
        print(f"Executing command: {' '.join(cmd)}")

        result = subprocess.run(cmd, capture_output=True, text=True)
        print(f"Command completed with return code: {result.returncode}")

        print(f"Command output:\n{result.stdout}")
        if result.returncode != 0:
            print(f"Command error output:\n{result.stderr}")

        os.remove(temp_file_path)
        debug_info = {
            "command": ' '.join(cmd),
            "return_code": result.returncode,
            "stdout": result.stdout,
            "stderr": result.stderr
        }
        print(f"Debug Info: {debug_info}")

        return {
            "success": result.returncode == 0,
            "errors": result.stderr.splitlines() if result.returncode != 0 else [],
            "output": result.stdout
        }
    except Exception as e:
        print(f"Command '{command}' failed: {e}")
        return {
            "success": False,
            "errors": [str(e)],
            "output": ""
        }

def reformat_code(code: str) -> str:
    try:
        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(code.encode())
            temp_file_path = temp_file.name

        subprocess.run(["black", "--line-length", "79", temp_file_path], check=True)

        with open(temp_file_path, 'r') as formatted_file:
            formatted_code = formatted_file.read()

        os.remove(temp_file_path)

        return formatted_code
    except subprocess.CalledProcessError as e:
        print(f"Formatting failed with error: {e}")
        return code
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return code

def refine_edit(raw_edit: str) -> str:
    """
    Refines the raw edit by removing markdown code block delimiters,
    stripping line numbers, and removing invalid lines.
    Ensures proper indentation and formatting using a code formatter.

    Args:
        raw_edit: The raw edit containing line numbers and delimiters.

    Returns:
        A refined and properly formatted version of the edit.
    """
    try:
        print("[DEBUG] Refining the suggested edit...")

        # Remove the markdown code block delimiters (```python and ```)
        refined_edit = raw_edit.strip("```python").strip("```").strip()
        
        # Remove line numbers at the start of each line
        refined_lines = []
        for line in refined_edit.split("\n"):
            # Use regex to remove leading line numbers followed by `:`
            refined_line = re.sub(r"^\d+:\s*", "", line).strip()
            # Skip invalid or non-Python lines (e.g., dashed lines)
            if re.match(r"^\-+$", refined_line):  # Matches lines with only dashes
                continue
            refined_lines.append(refined_line)
        
        refined_edit = "\n".join(refined_lines)
        print("[DEBUG] Refined content before formatting:")
        print(refined_edit)

        # Reformat the refined content to ensure proper indentation
        formatted_edit = reformat_code(refined_edit)
        print("[DEBUG] Refined and formatted content:")
        print(formatted_edit)

        return formatted_edit
    except Exception as e:
        print(f"[ERROR] Failed to refine edit: {e}")
        return ""




def detect_line_range_from_suggested_edit(suggested_edit: str) -> str:
    """
    Detects the line range from the provided suggested edit.

    Args:
        suggested_edit: The suggested edit block with line numbers.

    Returns:
        The line range as a string (e.g., "49 to 60").
    """
    try:
        print("[DEBUG] Detecting line range from suggested edit...")
        # Extract line numbers from the suggested edit
        line_numbers = re.findall(r"^(\d+):", suggested_edit, re.MULTILINE)
        if not line_numbers:
            raise ValueError("No line numbers detected in the suggested edit.")

        # Convert to integers and determine the range
        start_line = int(min(line_numbers))
        end_line = int(max(line_numbers))
        print(f"[DEBUG] Detected line range: {start_line} to {end_line}")

        return f"{start_line} to {end_line}"
    except Exception as e:
        print(f"[ERROR] Failed to detect line range: {e}")
        return ""

def remove_lines_by_numbers(code_block: str) -> str:
    """
    Removes line numbers and markers (```python and ```) from a Python code block while maintaining indentation.

    Args:
        code_block (str): The input Python code block as a string.

    Returns:
        str: The updated code block without line numbers and markers.
    """
    import re

    # Split the code block into lines
    lines = code_block.splitlines()

    # Filter out lines that are Python block markers
    filtered_lines = [line for line in lines if not line.startswith("```")]

    # Remove line numbers while preserving indentation
    updated_lines = []
    for line in filtered_lines:
        match = re.match(r"^(\d+):(\s*)(.*)$", line)
        if match:
            # Extract indentation and content after the line number
            indentation = match.group(2)
            content = match.group(3)
            updated_lines.append(f"{indentation}{content}")
        else:
            # If no line number, keep the line as is
            updated_lines.append(line)

    # Join the lines back into a single block
    return "\n".join(updated_lines)


# --------------------------------------------------------------------------------------------------------------
def apply_edit(file_path: str, llm_fix: str) -> str:
    print("[DEBUG] Starting edit application...")
    os.chdir('/workspace')
    print("Current Working Directory:", os.getcwd())
    


    # Step 1: Parse the suggested edits to extract line numbers and content
    def parse_edits(llm_fix):
        lines = llm_fix.splitlines()
        edits = []
        for line in lines:
            if not line.startswith("```"):
                match = re.match(r"^(\d+):\s*(.*)$", line)
                if match:
                    line_number = int(match.group(1)) - 1  # Convert to 0-based index
                    content = match.group(2)
                    edits.append((line_number, content))
        return edits

    edits = parse_edits(llm_fix)
    if not edits:
        print("[ERROR] No edits found or parsed.")
        return "No edits found or parsed."

    # Read the existing content from the file and keep a backup
    with open(file_path, 'r') as file:
        original_file_lines = file.readlines()
        file_lines = original_file_lines.copy()

    # Step 2: Apply the edits to the file while preserving original indentation and empty lines
    try:
        print("[DEBUG] Applying edits...")
        for line_number, new_content in edits:
            if line_number < len(file_lines):
                original_indentation = len(file_lines[line_number]) - len(file_lines[line_number].lstrip())
                if file_lines[line_number].strip() == '':
                    file_lines[line_number] = '\n'  # Preserve the empty line
                else:
                    file_lines[line_number] = ' ' * original_indentation + new_content + '\n'
                print(f"[DEBUG] Replacing line {line_number + 1} with: {file_lines[line_number].rstrip()}")
            else:
                print(f"[ERROR] Line number {line_number + 1} out of range.")
                return "Line number out of range."

        print("[DEBUG] Edits applied, writing back to file...")
        with open(file_path, 'w') as file:
            file.writelines(file_lines)
        print("[DEBUG] File updated successfully.")

        # Step 3: Format the code with a linter
        subprocess.run(["black", file_path], check=True)
        print("[DEBUG] Linter completed successfully.")
        return f"Successfully edited {file_path} and formatted the file."
    except subprocess.CalledProcessError as e:
        # Linter failed, restore original file
        with open(file_path, 'w') as file:
            file.writelines(original_file_lines)
        error_message = f"Linter failed to format the file, original file restored: {e}"
        print("[ERROR]", error_message)
        return error_message
    except Exception as e:
        error_message = f"Failed to edit {file_path}: {str(e)}"
        print("[ERROR]", error_message)
        return error_message

## Parser SandBox

In [38]:

def parse_llm_response_to_json(response: str) -> Dict[str, Any]:
    """Parse the LLM response into a structured JSON object with file grouping."""
    parsed_result = {"file_groups": {}}
    
    try:
        pattern = r"\*\*File:?\*\*:?\s*[`]?([^`\n]+)[`]?.*?\*\*Function:?\*\*:?\s*[`]?([^`\n]+)[`]?.*?\*\*Line Range:?\*\*:?\s*(\d+)\s*to\s*(\d+).*?\*\*Reasoning:?\*\*:?\s*((?:(?!\*\*File|\d+\.\s+\*\*File)[\s\S])*)"
        
        functions = re.findall(pattern, response, re.DOTALL)
        
        for file_path, function_name, line_start, line_end, reasoning in functions:
            file_path = file_path.strip()
            if file_path not in parsed_result["file_groups"]:
                parsed_result["file_groups"][file_path] = []
                
            parsed_result["file_groups"][file_path].append({
                "function": function_name.strip(),
                "line_range": {
                    "start": int(line_start),
                    "end": int(line_end)
                },
                "reasoning": reasoning.strip()
            })
            
    except Exception as e:
        print(f"Error parsing response: {e}")
        
    return parsed_result




# Example Usage
response = """
Based on the provided information and the analysis of the `separability_matrix` function and its related components, here are the details of the problematic function:
### Problematic Function Details
1. **File**: `/workspace/test-repos/astropy/astropy/modeling/separable.py`
2. **Function**: `separability_matrix`
3. **Line Range**: 66 to 102
   ```python
   def separability_matrix(transform):
       \"""
       Compute the correlation between outputs and inputs.
       ...
       \"""
       if transform.n_inputs == 1 and transform.n_outputs > 1:
           return np.ones((transform.n_outputs, transform.n_inputs), dtype=np.bool_)
       separable_matrix = _separable(transform)
       separable_matrix = np.where(separable_matrix != 0, True, False)
       return separable_matrix
   ```
4. **Reasoning**: 
   - The `separability_matrix` function is responsible for computing the separability between outputs and inputs.
   - The function calls `_separable(transform)` which seems to be crucial in determining the separability.
   - The problem may lie in how the function interprets compound models.
### Additional Note
If needed, further investigation into the `_separable` function may be warranted."""



# Example Usage
response2 = """
Based on the analysis of the `ascii.qdp` format issue, the following functions in the `qdp.py` file are likely related to the problem:

### Problematic Functions

1. **File:** `/workspace/test-repos/astropy/astropy/io/ascii/qdp.py`
   - **Function:** `_line_type`
   - **Line Range:** 20 to 86
   - **Reasoning:** This function interprets a single line of the QDP file and determines its type (e.g., command, data, comment). The function uses regular expressions to match expected patterns, including the assumption that commands are uppercase (e.g., "READ SERR"). Since the function raises a `ValueError` for lines that do not match the expected uppercase pattern, it directly contributes to the issue when encountering lowercase commands.

2. **File:** `/workspace/test-repos/astropy/astropy/io/ascii/qdp.py`
   - **Function:** `_get_lines_from_file`
   - **Line Range:** 135 to 146
   - **Reasoning:** This function reads lines from a QDP file, which are then processed by other functions. While it does not directly handle the case sensitivity, it plays a role in reading the lines that will be passed to `_line_type`, thus indirectly contributing to the issue.

3. **File:** `/workspace/test-repos/astropy/astropy/io/ascii/qdp.py`
   - **Function:** `_read_table_qdp`
   - **Line Range:** 385 to 422
   - **Reasoning:** This function reads a table from a QDP file and calls `_get_tables_from_qdp_file`, which processes the lines. If any of those lines are not recognized due to case sensitivity (as indicated in the issue), it will lead to the error described. The function is directly involved in the reading process, making it relevant to the problem.

These functions collectively contribute to the issue of case sensitivity in recognizing QDP commands, leading to the failure when trying to read QDP files with lowercase commands."""

# result = parse_llm_response_to_json(response)
# result2 = parse_llm_response_to_json(response2)


# print(json.dumps(result, indent=2))
# print(json.dumps(result2, indent=2))



def extract_json_from_block(text):
   """Extract JSON content between code block markers."""
   pattern = r"```json\s*(.*?)\s*```"
   match = re.search(pattern, text, re.DOTALL)
   
   if match:
       json_str = match.group(1)
       try:
           json_data = json.loads(json_str)
           result = {"file_groups": {}}
           
           for item in json_data:
               file_path = item["File"]
               if file_path not in result["file_groups"]:
                   result["file_groups"][file_path] = []
                   
               result["file_groups"][file_path].append({
                   "function": item["Function"],
                   "line_range": {
                       "start": int(item["Line Range"].split(" to ")[0]),
                       "end": int(item["Line Range"].split(" to ")[1])
                   },
                   "reasoning": item["Reasoning"]
               })
           
           return result
       except json.JSONDecodeError:
           return {"file_groups": {}}
   return {"file_groups": {}}



# Example Usage
response4 = """
```json
[
    {
        "File": "astropy/io/ascii/qdp.py",
        "Function": "_line_type",
        "Line Range": "20 to 49",
        "Reasoning": "This function is responsible for interpreting a line in the QDP file. The issue could be related to how it identifies command lines, which are currently expected to be in uppercase."
    },
    {
        "File": "astropy/io/ascii/qdp.py",
        "Function": "_interpret_err_lines",
        "Line Range": "149 to 178",
        "Reasoning": "This function deals with interpreting error lines from the 'READ SERR' and 'READ TERR' commands. The problem might be related to its handling of case sensitivity."
    },
    {
        "File": "astropy/io/ascii/qdp.py",
        "Function": "_read_table_qdp",
        "Line Range": "385 to 414",
        "Reasoning": "This function is responsible for reading a table from a QDP file. The error in reading the QDP file could be originating from this function if it does not handle case insensitivity properly."
    }
]
```
"""
# print(json.dumps(extract_json_from_block(response4), indent=4))



def extract_code_block(text):
   # Extract code with line numbers and indentation
   pattern = r'(\d+:\s*.*?)(?=\d+:|$)'
   matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL) 
   return '\n'.join(m.strip() for m in matches)

# Test usage
text = '''
<think>
Alright, I'm trying to fix this issue where the QDP file parser in Astropy isn't recognizing lowercase commands. Let me think through how to approach this.

First, the problem is that the current regular expression for matching commands in the `_interpret_line` function only matches uppercase letters. So, if a command is written in lowercase, like 'abort', it's not being recognized, leading to a ValueError.

I need to modify the regex to be case-insensitive so it can handle both uppercase and lowercase commands. How do I make a regex case-insensitive in Python? Oh right, I can use the re.IGNORECASE flag or make the pattern itself case-insensitive by including [A-Za-z] instead of just [A-Z].

Looking at the current code, the regex for commands is r'^([A-Z]+)\s*(.*)$'. To make it case-insensitive, changing it to r'^([A-Za-z]+)\s*(.*)$' should work. Alternatively, I could add the re.I flag, but modifying the pattern might be clearer.

Wait, using [A-Za-z] ensures that both cases are matched without relying on flags, which might be better for clarity. So I'll update the regex pattern to include both uppercase and lowercase letters.

Next, I need to update the code accordingly. The line where the regex is defined needs to be changed. Let me check the line number in the original code snippet provided. It looks like the regex is on line 45.

So, the corrected line would be: cmd_re = re.compile(r'^([A-Za-z]+)\s*(.*)$')

I should also consider if there are other places in the code that assume commands are uppercase. But from the given context, it seems the main issue is the regex, so changing that should suffice.

Testing this change would involve running the parser with lowercase commands to ensure they're correctly identified and processed without raising an error.
</think>

Here is the corrected code for the specified file:

```python
20: def _interpret_line(line):
21:     """
22:     Interpret a line in a QDP file.
23:     """
24:     # Strip whitespace from the line
25:     stripped_line = line.strip()
26:     if not stripped_line:
27:         return None, None
28:     # Check for comment line
29:     if stripped_line.startswith('!'):
30:         return 'comment', stripped_line
'''

# print(extract_code_block(text))


## General Sandbox

In [39]:
# def reset_and_test():
#     directory.cwd = "/"  # Reset to root
#     start_line = 20  # Define the start line
    
#     # Go to astropy directory first
#     goto_directory.invoke({"path": "astropy"})
    
#     original_code = open_file.invoke({
#         "path": "astropy/io/ascii/qdp.py",  # Relative path from within repo
#         "line_number": start_line,
#         "max_lines": 100
#     })
    
#     print("Original code extracted is:")
#     print(original_code)
#     print("=====================================")
#     print("Current directory:", get_current_dir.invoke({}))

# # reset_and_test()

## Structure Script

In [40]:
def extract_solved_issues(state: PatchState):
    print("\n------------------------------")
    print("Extracting issues")
    print("------------------------------")
   
    solved_issues_summary = {}
    
    try:
        diagnosis_str = state.get("diagnosis", "{}")
        diagnosis = json.loads(diagnosis_str) if isinstance(diagnosis_str, str) else diagnosis_str
        file_groups = diagnosis.get("file_groups", {}) if diagnosis else {}
        
        for file_path, issues in file_groups.items():
            if not isinstance(issues, list):
                print(f"[WARNING] Invalid issues format for {file_path}")
                continue
                
            solved_issues_summary[file_path] = []
            
            for issue in issues:
                try:
                    issue_data = {
                        "function": issue.get("function", "Unknown function"),
                        "line_range": f"{issue.get('line_range', {}).get('start', '?')}-{issue.get('line_range', {}).get('end', '?')}",
                        "issue": issue.get("reasoning", "No reasoning provided"),
                        "suggested_fix": issue.get("suggested_fix", "No fix provided"),
                        "edit_status": issue.get("edit_status", "Not edited")
                    }
                    
                    print(f"\nFile: {file_path}")
                    print(f"  Function: {issue_data['function']}")
                    print(f"  Line Range: {issue_data['line_range']}")
                    print(f"  Issue: {issue_data['issue'][:100]}...")
                    
                    if issue_data['suggested_fix']:
                        fix_preview = str(issue_data['suggested_fix'])[:100].replace('\n', ' ').strip()
                        print(f"  Suggested Fix: {fix_preview}...")
                    
                    solved_issues_summary[file_path].append(issue_data)
                    
                except Exception as e:
                    print(f"[ERROR] Failed to process issue: {str(e)}")
                    continue
                    
    except Exception as e:
        print(f"[ERROR] Failed to extract issues: {str(e)}")
        
    return solved_issues_summary

def save_patch_and_traces(output: PatchState, instance_id: str, detailed_log: dict, solved_issues: dict):
    print("\n------------------------------")
    print("Processing Traces")
    print("------------------------------")

    try:
        grouped_trajs = {"files": []}
        
        for file_path, issues in solved_issues.items():
            file_entry = {"file_name": file_path, "issues": []}
            
            for issue in issues:
                try:
                    suggested_fix = issue.get("suggested_fix", "No fix provided")
                    fix_code = (suggested_fix.get("fix_code", suggested_fix) 
                              if isinstance(suggested_fix, dict) else suggested_fix)
                    
                    file_entry["issues"].append({
                        "line_range": issue.get("line_range", "Unknown range"),
                        "description": issue.get("issue", "No description"),
                        "suggested_fix": fix_code or "No fix provided",
                        "solved": bool(output.get("patch", "").strip())
                    })
                except Exception as e:
                    print(f"[ERROR] Failed to process issue in traces: {str(e)}")
                    continue
                    
            grouped_trajs["files"].append(file_entry)

        traj_file_path = f"outputs/trajs/{instance_id}.json"
        with open(traj_file_path, "w") as traj_file:
            json.dump(grouped_trajs, traj_file, indent=2)
        print(f"[INFO] Reasoning traces saved to: {traj_file_path}")
        detailed_log["steps"].append("Reasoning traces saved.")
        
    except Exception as e:
        print(f"[ERROR] Failed to save traces: {str(e)}")
        detailed_log["steps"].append(f"Failed to save traces: {str(e)}")

def process_output(output: PatchState, instance_id: str, logs: dict, detailed_log: dict):
    print("\n------------------------------")
    print("Processing Output")
    print("------------------------------")
    
    try:
        solved_issues = extract_solved_issues(output)
        detailed_log["solved_issues"] = solved_issues
        
        for file_path, issues in solved_issues.items():
            for issue in issues:
                print(f"File: {file_path}")
                print(f"  Line Range: {issue.get('line_range')}")
                print(f"  Status: {issue.get('edit_status')}")
        
        if output.get("patch", "").strip():
            logs["resolved_count"] = logs.get("resolved_count", 0) + 1
            logs["resolved"] = logs.get("resolved", []) + [instance_id]
            detailed_log["steps"].append("Patch applied successfully")
        else:
            logs["resolved_with_errors_count"] = logs.get("resolved_with_errors_count", 0) + 1
            logs["resolved_with_errors"] = logs.get("resolved_with_errors", []) + [instance_id]
            detailed_log["steps"].append("Patch application encountered errors")

        logs["generated"] = logs.get("generated", 0) + 1
        
        save_patch_and_traces(output, instance_id, detailed_log, solved_issues)
        append_to_preds_json(instance_id, output)
        append_to_preds_jsonl(instance_id, output)
        
    except Exception as e:
        print(f"[ERROR] Failed to process output: {str(e)}")
        detailed_log["steps"].append(f"Output processing failed: {str(e)}")

def append_to_preds_json(instance_id: str, output: PatchState):
    try:
        prediction_entry = {
            "instance_id": instance_id,
            "model_name_or_path": "QUEN with GPT-4o",
            "model_patch": output.get("patch", "")
        }
        
        all_preds_file = "outputs/all_preds.json"
        all_preds = []
        
        if os.path.exists(all_preds_file):
            with open(all_preds_file, "r") as f:
                all_preds = json.load(f)
                
        all_preds.append(prediction_entry)
        
        with open(all_preds_file, "w") as f:
            json.dump(all_preds, f, indent=2)
            
        print(f"[INFO] Prediction appended to {all_preds_file}")
        
    except Exception as e:
        print(f"[ERROR] Failed to append prediction to JSON: {str(e)}")

def append_to_preds_jsonl(instance_id: str, output: PatchState):
    try:
        prediction_entry = {
            "instance_id": instance_id,
            "model_name_or_path": "QUEN with GPT-4o",
            "model_patch": output.get("patch", "")
        }
        
        with open("outputs/all_preds.jsonl", "a") as f:
            json.dump(prediction_entry, f)
            f.write("\n")
            
        print(f"[INFO] Prediction appended to all_preds.jsonl")
        
    except Exception as e:
        print(f"[ERROR] Failed to append prediction to JSONL: {str(e)}")

## Submission Script

In [41]:
# def extract_solved_issues(state: PatchState):
#     """
#     Extract and display solved issues from the diagnosis in the state.

#     Args:
#         state (PatchState): The state containing diagnosis and applied fixes.

#     Returns:
#         dict: Summary of resolved issues in a structured format.
#     """
#     solved_issues_summary = {}
#     diagnosis = state.get("diagnosis", {})
#     file_groups = diagnosis.get("file_groups", {})

#     # print("\n--- Solved Issues Report ---")
#     for file_path, issues in file_groups.items():
#         solved_issues_summary[file_path] = []
#         print(f"\nFile: {file_path}")
#         for issue in issues:
#             suggested_fix = issue.get("suggested_fix", None)
#             status = issue.get("edit_status", "Not Edited")
#             fix_code = suggested_fix.get("fix_code", "No code provided") if isinstance(suggested_fix, dict) else suggested_fix

#             print(f"  - Lines: {issue['lines']}")
#             print(f"    Issue: {issue['issue']}")
#             print(f"    Status: {status}")
#             print(f"    Suggested Fix: {fix_code if fix_code else 'Not Available'}")

#             solved_issues_summary[file_path].append({
#                 "lines": issue["lines"],
#                 "issue": issue["issue"],
#                 "status": status,
#                 "suggested_fix": fix_code or "No fix provided"
#             })

#     # print("\n--- End of Report ---")
#     return solved_issues_summary



# def save_patch_and_traces(output, instance_id, detailed_log, solved_issues):
#     """
#     Save patch and reasoning traces for a given instance in the SWE format.

#     Args:
#         output (PatchState): The workflow output containing patch and messages.
#         instance_id (str): Unique identifier for the instance.
#         detailed_log (dict): Log dictionary to append steps.
#         solved_issues (dict): Solved issues extracted from the output.
#     """
#     print("\n------------------------------")
#     print(f"Processing Traces")
#     print("------------------------------")

#     grouped_trajs = {"files": []}

#     # Add solved issues to the grouped_trajs
#     for file_path, issues in solved_issues.items():
#         file_entry = {"file_name": file_path, "issues": []}
#         for issue in issues:
#             suggested_fix = issue.get("suggested_fix", "No fix provided")
#             fix_code = suggested_fix.get("fix_code", "No code provided") if isinstance(suggested_fix, dict) else suggested_fix

#             file_entry["issues"].append({
#                 "line_range": issue["lines"],
#                 "description": issue["issue"],
#                 "suggested_fix": fix_code or "No fix provided",
#                 "solved": bool(output.get("patch", "").strip())  # Mark as solved if patch exists
#             })
#         grouped_trajs["files"].append(file_entry)

#     # Write the reasoning traces to file
#     traj_file_path = f"outputs/trajs/{instance_id}.json"
#     try:
#         with open(traj_file_path, "w") as traj_file:
#             json.dump(grouped_trajs, traj_file, indent=2)
#         print(f"[INFO] Reasoning traces saved to: {traj_file_path}")
#         detailed_log["steps"].append("Reasoning traces saved.")
#     except Exception as e:
#         print(f"[ERROR] Failed to save reasoning traces: {e}")
#         detailed_log["steps"].append(f"Failed to save reasoning traces: {e}")

#     # Save the patch
#     # patch_file_path = f"outputs/logs/{instance_id}_patch.json"
#     # try:
#     #     with open(patch_file_path, "w") as patch_file:
#     #         json.dump({"patch": output["patch"]}, patch_file, indent=2)
#     #     print(f"[INFO] Patch saved to: {patch_file_path}")
#     #     detailed_log["steps"].append("Patch saved.")
#     # except Exception as e:
#     #     print(f"[ERROR] Failed to save patch: {e}")
#     #     detailed_log["steps"].append(f"Failed to save patch: {e}")




# def process_output(output, instance_id, logs, detailed_log):
#     """
#     Process the output from the workflow.

#     Args:
#         output (PatchState): The workflow output.
#         instance_id (str): Unique identifier for the instance.
#         logs (dict): Log structure for the run.
#         detailed_log (dict): Detailed log for the instance.
#     """
#     print("\n------------------------------")
#     print(f"Processing Output")
#     print("------------------------------")

#     # Extract solved issues
#     solved_issues = extract_solved_issues(output)
#     detailed_log["solved_issues"] = solved_issues
#     # print("Solved Issues:")
#     for file_path, issues in solved_issues.items():
#         print(f"File: {file_path}")
#         for issue in issues:
#             print(f"  Lines: {issue['lines']}, Issue: {issue['issue']}, Status: {issue['status']}")
#     detailed_log["steps"].append("Solved issues extracted and logged.")

#     # Check for patch to determine resolution status
#     if output.get("patch", "").strip():
#         logs["resolved_count"] += 1
#         logs["resolved"].append(instance_id)
#         detailed_log["steps"].append("Patch applied successfully.")
#     else:
#         logs["resolved_with_errors_count"] += 1
#         logs["resolved_with_errors"].append(instance_id)
#         detailed_log["steps"].append("Patch application encountered errors.")

#     logs["generated"] += 1

#     # Save patch and traces
#     save_patch_and_traces(output, instance_id, detailed_log, solved_issues)

#     # Append to both all_preds.json and all_preds.jsonl
#     append_to_preds_json(instance_id, output)
#     append_to_preds_jsonl(instance_id, output)

# def append_to_preds_json(instance_id, output):
#     """
#     Append the repository's patch details to the all_preds.json file.

#     Args:
#         instance_id (str): Unique identifier for the instance.
#         output (PatchState): The workflow output containing patch and model information.
#     """
#     all_preds_file = "outputs/all_preds.json"

#     # Prepare data for the new prediction entry
#     prediction_entry = {
#         "instance_id": instance_id,
#         "model_name_or_path": "QUEN with GPT-4o", 
#         "model_patch": output.get("patch", "")
#     }

#     # Append to the all_preds.json file
#     try:
#         # If the file exists, load existing data; if not, create an empty list
#         if os.path.exists(all_preds_file):
#             with open(all_preds_file, "r") as preds_file:
#                 all_preds = json.load(preds_file)
#         else:
#             all_preds = []

#         all_preds.append(prediction_entry)

#         with open(all_preds_file, "w") as preds_file:
#             json.dump(all_preds, preds_file, indent=2)
#         print(f"[INFO] Prediction entry appended to {all_preds_file}")
#     except Exception as e:
#         print(f"[ERROR] Failed to append to {all_preds_file}: {e}")

# def append_to_preds_jsonl(instance_id, output):
#     """
#     Append the repository's patch details to the all_preds.jsonl file.

#     Args:
#         instance_id (str): Unique identifier for the instance.
#         output (PatchState): The workflow output containing patch and model information.
#     """
#     all_preds_file = "outputs/all_preds.jsonl"

#     prediction_entry = {
#         "instance_id": instance_id,
#         "model_name_or_path": "QUEN with GPT-4o", 
#         "model_patch": output.get("patch", "")
#     }

#     try:
#         with open(all_preds_file, "a") as preds_file:
#             json.dump(prediction_entry, preds_file)
#             preds_file.write("\n")  
#         print(f"[INFO] Prediction entry appended to {all_preds_file}")
#     except Exception as e:
#         print(f"[ERROR] Failed to append to {all_preds_file}: {e}")





#============================================================================================




# Step 1: Fork and clone the repository
def fork_and_clone():
    # Forking the repository manually if not already done.
    # Clone the forked repository.
    if not os.path.exists(LOCAL_REPO_DIR):
        subprocess.run(
            ["git", "clone", "--depth", "1", f"https://github.com/{GITHUB_USERNAME}/experiments.git", LOCAL_REPO_DIR],
            check=True
        )
    os.chdir(LOCAL_REPO_DIR)
    subprocess.run(["git", "remote", "add", "upstream", REPO_URL], check=True)
    subprocess.run(["git", "fetch", "upstream"], check=True)
    subprocess.run(["git", "checkout", BRANCH_NAME], check=True)

# Step 2: Create a submission directory
def create_submission_directory():
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
    # Copy output files to submission directory
    for item in os.listdir(OUTPUT_DIR):
        source_path = os.path.join(OUTPUT_DIR, item)
        dest_path = os.path.join(SUBMISSION_DIR, item)
        if os.path.isdir(source_path):
            shutil.copytree(source_path, dest_path, dirs_exist_ok=True)
        else:
            shutil.copy2(source_path, dest_path)

# Step 3: Commit and create a pull request
def commit_and_push():
    subprocess.run(["git", "add", "."], check=True)
    subprocess.run(["git", "commit", "-m", f"Add {SUBMISSION_NAME} submission"], check=True)
    subprocess.run(["git", "push", "origin", BRANCH_NAME], check=True)
    print("Your submission is ready. Create a pull request from your forked repository.")




# Agents

### LLM suggestion Agent

### Code Editor Agents

In [59]:
def get_fix_from_llm(file_path: str, line_range: str, issue_description: str, repo: str, function:str) -> str:
    """
    Fetches file content and invokes the editor LLM to suggest edits.

    Args:
        file_path: Path to the file.
        line_range: Line range to edit (e.g., "50 to 70").
        issue_description: Description of the issue.

    Returns:
        Suggested corrected code block.
    """
    print(f"\n[DEBUG] Invoking get_fix_from_llm")
    print(f"File Path: {file_path}")

    try:
        start_line, end_line = map(int, line_range.split(" to "))
        directory.cwd = "/"
        start_line = 20
        print("repository", repo)

        goto_directory.invoke({"path": repo})

        original_code = open_file.invoke({
            "path": file_path,
            "line_number": start_line,
            "max_lines": 100
        })

        current_dir = get_current_dir.invoke({})

        # Prepare messages for the LLM
        prompt = f"""Review and fix the following code issue write code in below format only once and  keep the thinking minimum:


Required Response Format:
```python
20: def example():  # Must keep original line numbers
21:     # Must preserve exact indentation
```

Line Range: {line_range}
Issue Description: {issue_description}
function: {function}


Original Code:
{original_code}



Strict Guidelines :
1. Code block must match exact line numbers
2. Preserve all whitespace/indentation
3. rewrite whole fucntion again that is provided and fualty if not there then focus on line range
4. Please dont Include any Imports assume they are already imported
"""

        messages = [
            {"role": "system", "content": "You are Qwen, created by to Perform To just give remedies in form of code block."},
            {"role": "user", "content": prompt}
        ]
        # print("=====================================")
        # print("Messages :", messages)
        # print("=====================================")

        # Generate the text
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        outputs = llm.generate([text], sampling_params)

        # Extract the raw response
        for output in outputs:
            prompt = output.prompt
            generated_text = output.outputs[0].text
            print(f"Generated text: {generated_text!r}")

        print(f"[DEBUG] Full LLM Response:\n{outputs[0].outputs[0].text}")
        print(f"[DEBUG] Code Block:\n{extract_code_block(outputs[0].outputs[0].text)}")
        print(f"[DEBUG]-----------------------------------------")

        return extract_code_block(outputs[0].outputs[0].text)

    except Exception as e:
        print(f"[ERROR] Exception in get_fix_from_llm: {e}")
        return "Error: Unable to generate suggested edits."



# using 4o
# def get_fix_from_llm(file_path: str, line_range: str, issue_description: str, repo: str, function:str) -> str:
#     """
#     Fetches file content and invokes the editor LLM to suggest edits.

#     Args:
#         file_path: Path to the file.
#         line_range: Line range to edit (e.g., "50 to 70").
#         issue_description: Description of the issue.

#     Returns:
#         Suggested corrected code.
#     """
#     print(f"\n[DEBUG] Invoking get_fix_from_llm")
#     print(f"File Path: {file_path}")
#     print(f"Line Range: {line_range}")
#     print(f"Issue Description: {issue_description}")

#     try:
#         start_line, _ = map(int, line_range.split(" to "))
#         # print(f"[DEBUG] Start Line: {start_line}")

#         # Fetch content using open_file tool
#         original_code = open_file.invoke({
#             "path": file_path,
#             "line_number": start_line,
#             "max_lines": 100
#         })
#         # print(f"[DEBUG] Original Code Fetched:\n{original_code}")

#         current_dir = get_current_dir.invoke({})
#         # print(f"[DEBUG] Current Directory: {current_dir}")

#         # Format the prompt
#         prompt = editor_prompt_template.format_messages(
#             file_path=file_path,
#             current_dir=current_dir,
#             line_range=line_range,
#             issue_description=issue_description,
#             original_code=original_code
#         )
#         # print(f"[DEBUG] Prompt Formatted:\n{prompt}")

#         # Invoke editor_llm
#         print("[DEBUG] Invoking editor_llm...")
#         response = editor_llm.invoke(prompt)
#         # print(f"[DEBUG] LLM Response:\n{response.content.strip()}")

#         return response.content.strip()

#     except Exception as e:
#         print(f"[ERROR] Exception in get_fix_from_llm: {e}")
#         return "Error: Unable to generate suggested edits."



# get_fix_from_llm("astropy/io/ascii/qdp.py", "23 to 88", "This function interprets the type of a QDP line and raises a ValueError if the line is unrecognized. It uses a regular expression to match command lines, which assumes commands are in uppercase.", "astropy","_line_type")

In [43]:
def code_editor(state: PatchState):
    print("--------------------------------------------------------")
    print("\n[DEBUG] Starting Code Editor Execution...")

    diagnosis_str = state.get("diagnosis", "{}")  
    try:
        diagnosis = json.loads(diagnosis_str)  
    except json.JSONDecodeError:
        print("[ERROR] Failed to parse the diagnosis JSON.")
        return state

    print(f"Diagnosis: {diagnosis}")

    file_groups = diagnosis.get("file_groups", {})
    if not file_groups:
        print("[ERROR] No file groups found in diagnosis.")
        return state

    messages = []
    updated_file_groups = {}

    for file_path, issues in file_groups.items():
        abs_file_path = os.path.join("test-repos", state["repo"], file_path)
        print(f"\n[DEBUG] Processing File: {file_path}")
        print(f"Absolute File Path: {abs_file_path}")

        updated_issues = []
        for issue in issues:
            line_range = issue["line_range"]
            description = issue["reasoning"]
            function = issue["function"]
            line_range_str = f"{line_range['start']} to {line_range['end']}"
            repo = state["repo"]

            print("--------------------------------------------------------")
            print(f"\n[DEBUG] Issue Details:")
            print(f"Line Range: {line_range_str}")
            print(f"Issue Description: {description}")
            print(f"Function: {function}")

            # Step 1: Fetch LLM-suggested edits
            print("\n[Node] Getting Response from LLM...")
            suggested_code_block = get_fix_from_llm(file_path, line_range_str, description, repo,function)
            
            # Skip if no code block is suggested
            if not suggested_code_block:
                print(f"[WARNING] No code block suggested for file {file_path}.")
                updated_issue = {
                    **issue,
                    "suggested_fix": "",
                    "edit_status": "Skipped - No suggestion provided"
                }
                updated_issues.append(updated_issue)
                continue

            print(f"[DEBUG] Suggested Edit code block extracted:\n{suggested_code_block}")
            print("--------------------------------------------------------")

            # Step 2: Apply the suggested edit to the file
            print("\n[Node] Applying Fix...")
            edit_result = apply_edit(abs_file_path, suggested_code_block)
            print(f"[DEBUG] Edit Result: {edit_result}")
            print("--------------------------------------------------------")

            # Step 3: Handle validation and linting
            # if "Linter failed" in edit_result:
                # print("\n[DEBUG] Linter Validation Failed. Requesting LLM for Corrections...")
                # corrected_edit = get_fix_from_llm(
                #     file_path,
                #     line_range,
                #     "The previous edit caused syntax errors. Please correct it."
                # )
                # if corrected_edit:
                #     print(f"[DEBUG] Corrected Edit:\n{corrected_edit}")
                #     edit_result = apply_edit(abs_file_path, corrected_edit)

            # Update the issue with the LLM fix
            updated_issue = {
                **issue,
                "suggested_fix": suggested_code_block,
                "edit_status": edit_result
            }
            updated_issues.append(updated_issue)

            # Update messages with the result
            messages.append({
                "role": "tool",
                "content": (
                    f"File: {file_path}, "
                    f"Lines: {line_range}, "
                    f"Status: {'skipped' if not suggested_code_block else 'edited'}, "
                    f"Edit Result: {edit_result}"
                ),
                "tool_call_id": f"{file_path}:{line_range}" 
            })
            print("[DEBUG] Edit successfully added to messages.")

        updated_file_groups[file_path] = updated_issues

    # Generate the patch for the repository
    directory_cwd = f"/{state['repo']}/"
    repo_path = os.path.join("test-repos", state["repo"])
    patch_path = os.path.join(directory_cwd, 'patch.diff')
    try:
        state['patch'] = generate_patch(repo_path, patch_path)
        print("--------------------------------------------------------")
        print(f"[INFO] Patch generation completed for {state['repo']}")
        print(f"[INFO] Patch Content:\n{state['patch']}")
    except Exception as e:
        print(f"[ERROR] Failed to generate patch: {e}")
        state['patch'] = ""

    updated_state = {
        **state,
        "diagnosis": json.dumps({  
            **diagnosis,
            "file_groups": updated_file_groups
        }, indent=4),
        "messages": messages
    }

    print("[DEBUG] Code Editor Execution Completed.")
    return updated_state

### Patch Generator Agent

In [44]:
def generate_patch(repo_path: str, patch_path: str) -> str:
    """
    Generate a unified diff patch for the current state of the repository and return its content.

    Args:
        repo_path (str): Path to the repository.
        patch_path (str): Full path to save the patch file.

    Returns:
        str: The content of the generated patch.

    Raises:
        subprocess.CalledProcessError: If Git commands fail.
    """
    print("--------------------------------------------------------")
    print(f"[DEBUG] Generating patch for repository: {repo_path}")
    print(f"[DEBUG] Patch will be saved at: {patch_path}")

    try:
        os.makedirs(os.path.dirname(patch_path), exist_ok=True)
        subprocess.run(["git", "add", "."], cwd=repo_path, check=True)
        
        with open(patch_path, 'w') as patch_file:
            subprocess.run(["git", "diff", "--cached"], cwd=repo_path, stdout=patch_file, check=True)
        
        print(f"[INFO] Patch successfully saved to {patch_path}")

        # Read and return the content of the patch
        with open(patch_path, 'r') as patch_file:
            patch_content = patch_file.read()

        return patch_content

    except subprocess.CalledProcessError as e:
        print("[ERROR] Error while generating patch!")
        print(f"[DEBUG] Error Details: {e}")
        raise

### Issue Dignaosing Agent

In [53]:
def diagnose_issue(state: PatchState):
    messages = state['messages']
    response = diagnoser_llm.invoke(messages)
    
    if response.content.strip():
        print("------------------------------")
        print(f"Diagnosis from diagnosis_module: {(response.content)}")
        print("------------------------------")
        print("------------------------------")
        diagnosis_json= extract_json_from_block(response.content)
        diagnosis_json = json.dumps(diagnosis_json, indent=4)
        print(f"Diagnosis from diagnosis_module json: {diagnosis_json}")
        state["diagnosis"] = diagnosis_json
        print("------------------------------")
    else:
        print("------------------------------")
        print("LLM is Diagnosing")
        print("------------------------------")
    
    return {
        **state,
        "messages": [response],
        "search_required": 'tool_calls' in response.additional_kwargs
    }

def search_issue_code(state: PatchState):
    """Identify problematic functions in the codebase."""
    issue_description = state['messages'][-1].content
    prompt = search_issue_code_prompt.format(issue=issue_description)
    response = search_llm.invoke([prompt])
    
    return {
        **state,
        "messages": [response]
    }


def should_use_search_tool(state: PatchState):
    """Determine if the search tool should be used."""
    messages = state['messages']
    last_message = messages[-1]
    return 'continue' if 'tool_calls' in last_message.additional_kwargs else 'end'


def extract_content_from_files(state: PatchState):
    """Extract file contents based on identified problematic lines."""
    # message = state['messages'][-1]
    # directory.cwd = f"/{state['repo']}/"  
    # prompt = extract_content_from_files_prompt.format(files=message.content, current_dir=directory.cwd)
    # response = extractor_llm.invoke([prompt])
    
    # print(f"{state['data']['instance_id']}: Extracted content for error-related files.")
    return {
        **state,
        "messages": [response]
    }



def check_diagnosis_complete(state: PatchState):
    """Check if the diagnosis has been completed and stored in the state."""
    if "diagnosis" in state:
        return {"next_step": "yes"}
    else:
        return {"next_step": "no"}


def should_move_to_code_editor(state: PatchState):
    """Determine if the code editor can be used (i.e., if diagnosis is complete)."""
    if "diagnosis" in state:
        return "yes"
    return "no"




# Main

In [60]:
# Simplified Workflow
workflow = StateGraph(PatchState)

# Nodes
workflow.add_node("diagnose_issue", diagnose_issue)
workflow.add_node("search_tools", search_tool_node)
workflow.add_node("extract_content_from_files", extract_content_from_files)
workflow.add_node("extract_code_tools", extract_code_tool_node)
workflow.add_node("code_editor", code_editor)

# workflow.add_node("code_editor", code_editor)

# Entry Point
workflow.set_entry_point("diagnose_issue")
workflow.add_conditional_edges(
    "diagnose_issue",
    should_use_search_tool,
    {
        "continue": "search_tools",
        "end": "code_editor",
    }
)
workflow.add_edge("search_tools", "diagnose_issue")



# Edge: Diagnose -> Code Editor
# workflow.add_edge("extract_content_from_files", "code_editor")

# Edge: Code Editor -> End

# Compile Workflow
app = workflow.compile()

# Render Workflow Graph
from IPython.display import Image, display
# display(Image(app.get_graph().draw_mermaid_png()))

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    pass

# ===================================================
# ============================
# Folder Structure Initialization
# ============================
initial_cwd = os.getcwd()
os.makedirs(os.path.join(initial_cwd, "outputs/trajs"), exist_ok=True)
# os.makedirs(os.path.join(initial_cwd, "outputs/logs"), exist_ok=True)

# ============================
# Metadata Initialization and Configuration
# ============================
GITHUB_USERNAME = "shaafsalman"  
REPO_URL = "https://github.com/swe-bench/experiments.git"
BRANCH_NAME = "main"
SUBMISSION_NAME = "20250119_auto-repo" 
LOCAL_REPO_DIR = "./experiments"
OUTPUT_DIR = "./output"
TARGET_SPLIT = "evaluation/lite"
SUBMISSION_DIR = os.path.join(LOCAL_REPO_DIR, TARGET_SPLIT, SUBMISSION_NAME)
metadata = {
    "name": "auto-repo",
    "oss": True,
    "site": "https://github.com/abideenml",
    "verified": False
}
metadata_file_path = "outputs/metadata.yaml"
with open(metadata_file_path, "w") as metadata_file:
    yaml.dump(metadata, metadata_file, default_flow_style=False)
print("[INFO] Metadata saved.")

# ============================
# Dataset Configuration
# ============================
# dataset = load_dataset(path="ahsanirfan961/swe-bech-lite-bm25-13k-take50", split="train")
# dataset = load_dataset('princeton-nlp/SWE-bench_Verified', split='test')
dataset = load_dataset('princeton-nlp/SWE-bench_Lite', split='test')
MAX_REPOS = 1

# ============================
# Logs and Configuration
# ============================

predictions = []
logs = {
    "total": min(len(dataset), MAX_REPOS),
    "generated": 0,
    "resolved_count": 0,
    "resolved_with_errors_count": 0,
    "resolved": [],
    "resolved_with_errors": [],
    "generation_errors": [],
    "generation_time_in_seconds": [],
    "detailed_logs": []
}

print(f"Generating predictions for up to {MAX_REPOS} repositories")
for i, data in enumerate(dataset):
    if i >= MAX_REPOS:
        print("Reached maximum repository limit. Stopping...")
        break

    start_time = time()
    detailed_log = {
        "instance_id": data["instance_id"],
        "repo_name": data["repo"],
        "steps": []
    }
    print("\n------------------------------")
    print(f"Processing Repository {i + 1}/{logs['total']}")
    print("------------------------------")

    try:
        print(f"Setting up repository: {data['repo']}")
        repo_name = data["repo"].split("/")[-1]
        setup_repo(data)
        detailed_log["steps"].append("Repository setup completed.")

        # Prepare initial state
        initial_state = {
            "messages": [HumanMessage(content=search_issue_code_prompt.format(repo=repo_name,cwd=directory.cwd,issue=data['problem_statement']))],
            "patch": "",
            "code": "",
            "example_patch": "",
            "patch_application_failures": 0,
            "repo": repo_name,
            "data": data,
            "diagnosis": {},
            "failed_tests": data.get("FAIL_TO_PASS", [])
        }
        # print("\n------------------------------")
        # print(f'Problem Statement {i + 1}/{data["problem_statement"]}')
        # print("------------------------------")

        # Run the workflow
        output: PatchState = app.invoke(initial_state, config=RunnableConfig(recursion_limit=100))
        detailed_log["steps"].append("Workflow invoked.")

        # Process the output
        process_output(output, data["instance_id"], logs, detailed_log)

    except Exception as e:
        logs["generation_errors"].append({
            "instance_id": data["instance_id"],
            "error": str(e)
        })
        detailed_log["steps"].append(f"Error encountered: {str(e)}")
        print(f"[ERROR] {str(e)}")

    end_time = time()
    logs["generation_time_in_seconds"].append({
        "instance_id": data["instance_id"],
        "time": end_time - start_time
    })
    detailed_log["steps"].append(f"Time taken: {end_time - start_time} seconds.")
    logs["detailed_logs"].append(detailed_log)
    print("------------------------------")
    print(f"Finished processing repository {data['repo']}")
    print("------------------------------\n")
    print(f"Cleaning up repository: {repo_name}")
    repo_dir = os.path.join("test-repos", repo_name)
    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
        print(f"Repository {repo_name} removed from 'test-repos'.")


# Submission to SWE
# ============================
# try:
#     fork_and_clone()
#     create_submission_directory()
#     commit_and_push()
# except Exception as e:
#     print(f"An error occurred: {e}")


# log_file_path = "outputs/logs/logs.json"
# with open(log_file_path, "w") as log_file:
#     json.dump(logs, log_file, indent=2)


[INFO] Metadata saved.
Generating predictions for up to 1 repositories

------------------------------
Processing Repository 1/1
------------------------------
Setting up repository: astropy/astropy
Executing command:
 ['cd test-repos', 'git config --global http.postBuffer 524288000', 'git clone https://github.com/astropy/astropy.git']
Cloning into 'astropy'...
Updating files: 100% (1849/1849), done.

Executing command:
 ['cd test-repos/astropy', 'git checkout -b astropy__astropy-12907 d16bfe05a744909de4b27f5875fe0d4ed41ce607']
Switched to a new branch 'astropy__astropy-12907'

Repo setup complete
------------------------------
LLM is Diagnosing
------------------------------
------------------------------
LLM is Diagnosing
------------------------------
------------------------------
LLM is Diagnosing
------------------------------
Opening file: astropy/separable.py from line 27 with max 50 lines
Opening file: astropy/test_separable.py from line 59 with max 50 lines
------------------


Processed prompts: 100%|██████████| 1/1 [01:28<00:00, 88.11s/it, est. speed input: 16.68 toks/s, output: 22.05 toks/s]
error: cannot format test-repos/astropy/astropy/modeling/separable.py: Cannot parse: 316:0: EOF in multi-line statement

Oh no! 💥 💔 💥
1 file failed to reformat.


Generated text: 'Alright, I need to review and fix the code for the `separability_matrix` function in the Astropy modeling module. The function is supposed to compute the correlation between outputs and inputs of a transformation model, returning a boolean matrix indicating which outputs depend on which inputs.\n\nFirst, I need to understand what the function is supposed to do. It takes a model, which could be a simple or compound model, and returns a matrix where each row corresponds to an output and each column to an input. A True value indicates that the output depends on that input, and False indicates no dependence.\n\nLooking at the code, it seems there\'s a special case when the model has only one input but multiple outputs. In that case, it returns a matrix of all True, assuming all outputs depend on the single input.\n\nFor other cases, it calls a helper function `_separable(transform)` to get some kind of matrix, and then converts it to a boolean matrix where non-zero values 


Processed prompts: 100%|██████████| 1/1 [01:25<00:00, 85.91s/it, est. speed input: 16.60 toks/s, output: 21.99 toks/s]


Generated text: 'Alright, I\'ve got this task to review and fix some code in a file called `core.py` from the Astropy modeling module. The specific function that\'s causing issues is `_calculate_separability_matrix`, and it\'s somewhere between lines 808 and 815. The problem seems to be that it\'s not handling nested CompoundModels correctly, which is leading to an incorrect separability matrix.\n\nFirst, I need to understand what a separability matrix is in this context. From what I recall, in modeling, especially when dealing with compound models that are combinations of simpler models, the separability matrix likely indicates which parameters affect which outputs independently. This is crucial for optimization and fitting procedures, as it can significantly impact the efficiency and correctness of the algorithms used.\n\nCompoundModels in Astropy are models that are composed of other models, either through arithmetic operations or by chaining them together in a pipeline. Nested Comp